# Train / Dev / test sets

> Building a NN is an iterative process

Stractured data is used for Ads, speach, sequirity, logistics.  
Thus coming from NLP, or vision, it is not strigtforward to go to analyzing stractured data. 

Data us usually split into 
- Test set 
- cross-validation set
- test set

Common rule is 70/30 or 60,20,10 for small data sets.  
In case of huge datasets: just a few percent for test sets are sufficient.  

> data from test and cv sets should be coming from **the same distribution**. 

Sometimes only test and cv sets are needed. So overall, generally, 2 datasets are the bare minimum needed. 

#### Bias vs Variance (underfitting vs overfitting)
Consider Train set error and dev set error. (assume that test error __should__ be very low i.e., base error is low)
- If they are $1$ and $11$, this is **high variance**. 
- If they are $11$ and $16$, this is **high bias**. 
- If they are $15$ and $30$, this is **high bias & varaince**. 
- If they are $.5$ and $1$, this is **low bias & varaince**. 

#### Improving high bias / varaince problem

Check the bias (looking for training set).  If bias is high, try other network configurations, ($L$, $\alpha$, ..., architecture)  
**Note** Getting more data here usually _does not_ help

After bias is solved, the high variance problem can be solved by 
(e.g., getting more data, regulariation, more appropriate NN achitecture). 

> Bias/variance tradeoff. Old proplem. Can be solved with Bigger NN (reduces bias) and getting more data (reduces variance)




### Recularzation

> Regularization prevents overfitting.

In, for example, logisitc regression, we are trying to minimze the cost function. In **Regularization** we add the regularization term to the cost
$$
J(w,b) += \frac{\lambda}{2m}b^2
$$
where the notm of $w^2$, the **$L_2$ regularization** term reads
$$
||w^{[l]}||^2 = \sum_{i=1}^{n^{[l]}}\sum_{j=1}^{n^{[l-1]}}( w_{i,j}^{[l]} )^2
$$
One can also consider $L_1$ regularization term $\lambda/m ||w||_1$
Here $w$ is sparse (has too many zeros), and can help with **compressing** the model. 

> $\lambda$, the regularization term, is set using CV set.

In NN, the regularization is added as ter to the cost function where one has to loop over all training examples and units
$$
||w^{[l]}||^2 = \sum_{i=1}^{n^{[l]}}\sum_{j=1}^{n^{[l-1]}}( w_{i,j}^{[l]} )^2
$$
> THis is so-called **Frobinious norm** of the matrix

In the past $dw$ was computed using backprop. Now the update is:
$$
dw^{[l]} = (\text{from backprop}) + \frac{\lambda}{m}w^{[l]};\,\, \frac{\partial J}{\partial w^{[l]}} = dw^{[l]}
$$
> Sometimes $L2$ regularization is called ''weight decay''